In [24]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle as pkl
from pymongo import MongoClient

MONGO_PORT = 27017



In [25]:
client = MongoClient('localhost', MONGO_PORT)
db = client['entrdir']
collection = db['employee']
employees = list(collection.find())
employees = pd.DataFrame(employees)
print(employees.head())

target_features_fields = employees[['salary','role','work_location']]
target_features_fields = target_features_fields.dropna()
target_features_fields = target_features_fields.drop_duplicates()


                        _id  id               name                 phone  \
0  6870211c7f33aff0d95f96a8   3    Gabrielle Davis          855-394-0781   
1  6870211c7f33aff0d95f96a9   2      Lance Hoffman  001-486-537-9402x654   
2  6870211c7f33aff0d95f96aa   6     Andrew Stewart      513-695-3767x242   
3  6870211c7f33aff0d95f96ab   5     Henry Santiago  001-719-228-3276x483   
4  6870211c7f33aff0d95f96ac   4  Sandra Montgomery     (993)710-3413x164   

              role work_location  salary  manager_id        manages  
0   Data Scientist      Hartford  120000        1233             []  
1   Data Scientist      New York  120000         179  [66, 62, 192]  
2  Sales Executive       Chicago  105000         319         [1961]  
3       HR Manager       Chicago   55000        1555         [1430]  
4       HR Manager      Hartford   95000         860          [920]  


In [26]:
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(target_features_fields[['role','work_location']])
X_encoded = pd.DataFrame(X_encoded)
y= target_features_fields['salary']
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(accuracy)




0.5869355613858929


In [27]:
with open('model_dump.pkl', 'wb') as f:
    pkl.dump(model, f)

with open('encoder_dump.pkl', 'wb') as f:
    pkl.dump(encoder, f)